In [1]:
init_jupyter()
import chdrft.sim.ofoam as ofoam
ctx = app.setup_jup(parser_funcs=[ofoam.args])
import sys
sys.path.append('/usr/share/gmsh/api/python/')
import gmsh
import fipy
import pymesh
import pygmsh
from scipy.spatial.transform import Rotation as R
%load_ext autoreload
%autoreload 2
import meshio

W1013 00:57:06.627876 1178384 __init__.py:121] Arrangement computation is unavailable


Got exception No module named 'unicorn'
Runnign magic qt5
done
False


--------------------------------------------------------------------------
The library attempted to open the following supporting CUDA libraries,
but each of them failed.  CUDA-aware support is disabled.
libcuda.so.1: cannot open shared object file: No such file or directory
libcuda.dylib: cannot open shared object file: No such file or directory
/usr/lib64/libcuda.so.1: cannot open shared object file: No such file or directory
/usr/lib64/libcuda.dylib: cannot open shared object file: No such file or directory
If you are not interested in CUDA-aware support, then run with
--mca opal_warn_on_missing_libcuda 0 to suppress this message.  If you are interested
in CUDA-aware support, then try setting LD_LIBRARY_PATH to the location
of libcuda.so.1 to get passed this issue.
--------------------------------------------------------------------------


In [2]:
import chdrft.utils.misc as cmisc
import numpy as np
cmisc.yaml_dump_custom(dict(abc=np.float64(1), xx=dict(ddd=33)))


'abc: 1.0\nxx:\n  ddd: 33\n'

In [44]:
def gen_cylinder(p0, axis, radius, fname):
    with pygmsh.occ.Geometry() as geom:
        geom.characteristic_length_max = 2.5
        geom.add_cylinder(p0, axis, radius)
        mesh = geom.generate_mesh()
        print(mesh)
        gmsh.write(fname)
        
def transform_mesh(mesh, func=None, mat=None):
    if func is None: func = lambda x: Z.opa_math.MatHelper.mat_apply_nd(mat, x.T, n=3, point=1).T
    return meshio.Mesh(func(mesh.points), mesh.cells)

def centered_rot(rmat, pos=[0,0,0]):
    pos = np.array(pos)
    mh = Z.opa_math.MatHelper
    return mh.mat_apply_nd(mh.mat_translate(pos), mh.mat4(rmat), mh.mat_translate(-pos))
def gen_rot_grid(npts):
    tb=[]
    for i in range(3):
        mul = [1,2][i!=0]
        tb.append(np.linspace(0, mul*np.pi, mul * npts, endpoint=False))
    
    return R.from_euler('ZYX', list(itertools.product(*tb)))

class AnnotatedFiles(Z.ExitStack):
    def __init__(self, path, key, w=0):
        super().__init__()
        cmisc.makedirs(path)
        self.path = path
        self.key = key
        self.id = 0
        self.entries = []
        self.conf_file = f'{self.path}/chx_conf_{self.key}.yaml'
        
    def __enter__(self):
        super().__enter__()
        self.callback(self.write)
        return self
    
    def load(self):
        conf = Z.FileFormatHelper.Read(self.conf_file)
        self.id = conf.id
        self.entries = conf.entries
        for x in self.entries: self.norm(x)
    def query(self):
        return cmisc.asq_query(self.entries)
        
    def write(self):
        Z.FileFormatHelper.Write(self.conf_file, A(entries=self.entries, id=self.id))
        
    def make_fname(self, data):
        return cmisc.yaml_dump_custom(data, default_flow_style=True).replace('/', '_').replace('\n', '').replace(' ', '')
        
    def add_file(self, data, ext=None):
        idx = self.id
        self.id += 1
        e =A(data=data, id=idx, fname=f'chx_file_{self.key}_{self.make_fname(data)}_{idx:04}{ext}')
        self.norm(e)
        self.entries.append(e)
        return e.path
        
    def norm(self, e):
        e.path = f'{self.path}/{e.fname}'
        return e.path
        

In [3]:
gen_cylinder([-10,0,0], [20,0,0], 3, 'test.stl')

<meshio mesh object>
  Number of points: 139
  Number of cells:
    line: 28
    triangle: 258
    tetra: 341
    vertex: 2


In [49]:
with  AnnotatedFiles('./data', 'rots') as af:
    mesh_in =meshio.read('test.stl')
    for rot in gen_rot_grid(3):
        p = af.add_file(A(rot=rot.as_euler('ZYX')), '.stl')
        mout = transform_mesh(mesh_in, mat=centered_rot(rot.as_matrix()))
        mout.write(p)
        

In [79]:
mesh_in =meshio.read('test.stl')
transform_mesh(mesh_in, mat=centered_rot(rot.as_matrix()))


<meshio mesh object>
  Number of points: 131
  Number of cells:
    triangle: 258

In [46]:

af = AnnotatedFiles('./data', 'rots')
af.load()
for x in af.entries:
    Z.shutil.copyfile(x.path, f'./tmp/t{x.id}.vtk')

In [39]:
list(Z.itertools.product(range(2), repeat=2))

[(0, 0), (0, 1), (1, 0), (1, 1)]

In [32]:
x.id

124

In [85]:
print(af.make_fname(dict(rot=rot.as_euler('ZYX')[0])))

rot: !!python_object_apply:numpy.core.multiarray.scalar
- !!python_object_apply:numpy.dtype
  args:
  - f8
  - false
  - true
  state: !!python_tuple
  - 3
  - <
  - null
  - null
  - null
  - -1
  - -1
  - 0
- !!binary |
  AAAAAAAAAAA=



In [75]:

import meshio
mo = meshio.Mesh(mesh_in.vertices, [('triangle', mesh_in.faces)])
mo.write('./test2.stl')

In [77]:
mo.cells

[<meshio CellBlock, type: triangle, num cells: 258>]